# 평가지표
- 정확도
- 모델 클래스는 직접 구현

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [2]:
DATA_PATH = "../data/pizza_steak_sushi/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 음식 분류 데이터셋
- 0 : 피자
- 1 : 스테이크
- 2 : 스시


In [3]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")
train.shape , test.shape

((1649, 2), (1350, 2))

In [4]:
train.head()

,file_name,target
0,2104569.jpg,0
1,2038418.jpg,1
2,1919810.jpg,2
3,2557340.jpg,0
4,3621562.jpg,1


In [5]:
test.head()

,file_name,target
0,3777020.jpg,NaN
1,931356.jpg,NaN
2,2599817.jpg,NaN
3,1251166.jpg,NaN
4,1183595.jpg,NaN


# 1. 데이터셋 클래스 만들기


In [6]:
import cv2
import matplotlib.pyplot as plt

In [7]:
train_file = train["file_name"].to_numpy()
test_file = test["file_name"].to_numpy()

train_file.shape, test_file.shape

((1649,), (1350,))

In [8]:
train_file[0]

'2104569.jpg'

In [9]:
train_target = train["target"].to_numpy()

train_target.shape

(1649,)

In [10]:
class FoodsDataset(torch.utils.data.Dataset):
    def __init__(self, x, y=None, resize=(224, 224)):
        self.x = x
        self.y = y
        self.resize = resize

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        item = {}
        x = cv2.imread(f"{DATA_PATH}data/{self.x[idx]}")
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        x = cv2.resize(x, self.resize)
        x = x / 255
        item["x"] = torch.Tensor(x)

        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])

        return item

In [11]:
dataset = FoodsDataset(train_file, train_target)
dataloader = torch.utils.data.DataLoader(dataset, 1)
batch = next(iter(dataloader))
batch["x"].shape

torch.Size([1, 224, 224, 3])

# 모델 클래스 만들기

In [12]:
class Conv2dNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.seq = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.PReLU(),
            torch.nn.MaxPool2d(2),
        )

    def forward(self, x):
        return self.seq(x)

In [13]:
class LinearNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.seq = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(in_channels, out_channels),
            torch.nn.BatchNorm1d(out_channels),
            torch.nn.PReLU(),
        )

    def forward(self, x):
        return self.seq(x)

In [14]:
class Net(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=16, kernel_size=3):
        super().__init__()
        self.seq = torch.nn.Sequential(
            Conv2dNet(in_channels, out_channels, kernel_size),
            Conv2dNet(out_channels, out_channels * 2, kernel_size),
            Conv2dNet(out_channels * 2, out_channels * 4, kernel_size),
            Conv2dNet(out_channels * 4, out_channels * 8, kernel_size),

            torch.nn.AdaptiveMaxPool2d(1),
            torch.nn.Flatten(),
            LinearNet(out_channels * 8, out_channels * 4),
            LinearNet(out_channels * 4, out_channels * 2),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(out_channels * 2, 3),
        )

    def forward(self, x):
        return self.seq(x.permute(0, 3, 1, 2))

# 2. 학습 loop 함수 만들기

In [15]:
def train_loop(dataloader, model, loss_function, optimizer, device):
    epoch_loss = 0
    model.train()

    for batch in dataloader:
        pred = model(batch["x"].to(device))
        loss = loss_function(pred, batch["y"].to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

# 3. 테스트 loop 함수 만들기
- 데이터 예측 기능 및 검증데이터 손실값 반환하는 기능

In [16]:
@torch.no_grad()
def test_loop(dataloader, model, loss_function, device):
    epoch_loss = 0
    model.eval()

    act = torch.nn.Softmax(dim=1)
    pred_list = []
    for batch in dataloader:
        pred = model(batch["x"].to(device))
        if batch.get("y") is not None:
            loss = loss_function(pred, batch["y"].to(device))
            epoch_loss += loss.item()

        pred = act(pred)
        pred = pred.to("cpu").numpy()
        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dataloader)

    return epoch_loss, pred

# 4. 학습하기


In [17]:
n_splits = 5
batch_size = 16
epochs = 100
loss_function = torch.nn.CrossEntropyLoss()

In [18]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

cv = KFold(n_splits, shuffle=True, random_state=SEED)

In [23]:
is_holdout = False
reset_seeds(SEED)
score_list = []

for i, (tri, vai) in enumerate(cv.split(train_file)):
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters())

    train_dataset = FoodsDataset(train_file[tri], train_target[tri])
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_dataset = FoodsDataset(train_file[vai], train_target[vai])
    valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    best_score = 0
    best_loss = np.inf
    patience = 0

    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dataloader, model, loss_function, optimizer, device)
        valid_loss, pred = test_loop(valid_dataloader, model, loss_function, device)
        pred = np.argmax(pred, axis=1)
        score = accuracy_score(train_target[vai], pred)

        print(f"train_loss: {train_loss: .4f}, valid_loss: {valid_loss: .4f}, score: {score: .4f}")
        patience += 1
        if score > best_score or best_loss > valid_loss:
            patience = 0
            best_score = score
            best_loss = valid_loss
            torch.save(model.state_dict(), f"../output/model_{i}.pt")

        if patience == 10:
            break

    print(f"Fold-{i} Best Acc: {best_score: .4f}")
    score_list.append(best_score)

    if is_holdout:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  1.0160, valid_loss:  0.8884, score:  0.5939
train_loss:  0.8747, valid_loss:  0.9694, score:  0.4970
train_loss:  0.7742, valid_loss:  1.0462, score:  0.4758
train_loss:  0.7343, valid_loss:  0.8373, score:  0.6212
train_loss:  0.7071, valid_loss:  0.7425, score:  0.7000
train_loss:  0.6567, valid_loss:  0.6852, score:  0.7030
train_loss:  0.5912, valid_loss:  0.7586, score:  0.6727
train_loss:  0.6136, valid_loss:  0.8424, score:  0.6576
train_loss:  0.5255, valid_loss:  0.5584, score:  0.7545
train_loss:  0.5171, valid_loss:  0.7969, score:  0.6697
train_loss:  0.4718, valid_loss:  0.7574, score:  0.6818
train_loss:  0.4819, valid_loss:  0.5388, score:  0.7909
train_loss:  0.4435, valid_loss:  0.7721, score:  0.7182
train_loss:  0.4475, valid_loss:  0.8627, score:  0.6545
train_loss:  0.4021, valid_loss:  0.5610, score:  0.8091
train_loss:  0.3791, valid_loss:  0.6137, score:  0.7788
train_loss:  0.3829, valid_loss:  0.6681, score:  0.7515
train_loss:  0.3677, valid_loss

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  1.0288, valid_loss:  0.9888, score:  0.4939
train_loss:  0.8870, valid_loss:  0.8699, score:  0.5848
train_loss:  0.8146, valid_loss:  0.7319, score:  0.6939
train_loss:  0.7336, valid_loss:  0.6662, score:  0.7455
train_loss:  0.6736, valid_loss:  1.1135, score:  0.5606
train_loss:  0.6620, valid_loss:  0.6510, score:  0.7242
train_loss:  0.6019, valid_loss:  0.7303, score:  0.6909
train_loss:  0.5719, valid_loss:  0.6635, score:  0.7061
train_loss:  0.5407, valid_loss:  1.1509, score:  0.5000
train_loss:  0.5035, valid_loss:  0.5057, score:  0.7727
train_loss:  0.4806, valid_loss:  0.5779, score:  0.7424
train_loss:  0.4706, valid_loss:  0.4465, score:  0.8212
train_loss:  0.4565, valid_loss:  0.5011, score:  0.8091
train_loss:  0.4146, valid_loss:  0.4375, score:  0.7970
train_loss:  0.4135, valid_loss:  0.4464, score:  0.8182
train_loss:  0.3977, valid_loss:  0.4654, score:  0.8030
train_loss:  0.3581, valid_loss:  0.5596, score:  0.7667
train_loss:  0.3655, valid_loss

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  1.0597, valid_loss:  0.9378, score:  0.5909
train_loss:  0.9179, valid_loss:  0.7963, score:  0.6909
train_loss:  0.8103, valid_loss:  0.7731, score:  0.6879
train_loss:  0.7540, valid_loss:  0.6943, score:  0.7000
train_loss:  0.7031, valid_loss:  0.7406, score:  0.6909
train_loss:  0.6561, valid_loss:  1.1478, score:  0.5909
train_loss:  0.6007, valid_loss:  0.6624, score:  0.7212
train_loss:  0.5831, valid_loss:  0.6616, score:  0.7273
train_loss:  0.5775, valid_loss:  0.7254, score:  0.6818
train_loss:  0.5106, valid_loss:  0.5916, score:  0.7788
train_loss:  0.5040, valid_loss:  0.5493, score:  0.8000
train_loss:  0.4598, valid_loss:  0.5606, score:  0.7788
train_loss:  0.4716, valid_loss:  0.4913, score:  0.8030
train_loss:  0.4336, valid_loss:  0.5885, score:  0.7636
train_loss:  0.4467, valid_loss:  0.7090, score:  0.7000
train_loss:  0.4167, valid_loss:  0.4685, score:  0.8121
train_loss:  0.3482, valid_loss:  0.9725, score:  0.6697
train_loss:  0.3602, valid_loss

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  1.0401, valid_loss:  0.9456, score:  0.5576
train_loss:  0.8943, valid_loss:  0.8095, score:  0.6545
train_loss:  0.8147, valid_loss:  0.7916, score:  0.6152
train_loss:  0.7226, valid_loss:  0.7654, score:  0.6697
train_loss:  0.6610, valid_loss:  0.8191, score:  0.6333
train_loss:  0.6543, valid_loss:  0.9629, score:  0.6030
train_loss:  0.6373, valid_loss:  1.0584, score:  0.5242
train_loss:  0.5900, valid_loss:  0.6015, score:  0.7545
train_loss:  0.5517, valid_loss:  0.6028, score:  0.7485
train_loss:  0.5491, valid_loss:  0.5055, score:  0.8152
train_loss:  0.5051, valid_loss:  0.5359, score:  0.7909
train_loss:  0.4716, valid_loss:  0.5932, score:  0.7485
train_loss:  0.4881, valid_loss:  0.5022, score:  0.8152
train_loss:  0.4618, valid_loss:  0.5950, score:  0.7727
train_loss:  0.4125, valid_loss:  0.6372, score:  0.7515
train_loss:  0.4048, valid_loss:  0.4339, score:  0.8485
train_loss:  0.4218, valid_loss:  0.6968, score:  0.7242
train_loss:  0.3601, valid_loss

  0%|          | 0/100 [00:00<?, ?it/s]

train_loss:  1.0404, valid_loss:  0.8518, score:  0.6353
train_loss:  0.8889, valid_loss:  0.7662, score:  0.6596
train_loss:  0.8244, valid_loss:  0.7138, score:  0.6778
train_loss:  0.7706, valid_loss:  0.6635, score:  0.7204
train_loss:  0.6787, valid_loss:  0.5579, score:  0.7568
train_loss:  0.6896, valid_loss:  0.9544, score:  0.5380
train_loss:  0.5992, valid_loss:  1.2173, score:  0.5502
train_loss:  0.5853, valid_loss:  0.4964, score:  0.8176
train_loss:  0.5779, valid_loss:  0.6766, score:  0.7082
train_loss:  0.5315, valid_loss:  0.6791, score:  0.6930
train_loss:  0.5032, valid_loss:  0.4376, score:  0.8146
train_loss:  0.4696, valid_loss:  0.8826, score:  0.6292
train_loss:  0.4326, valid_loss:  0.4982, score:  0.7964
train_loss:  0.4399, valid_loss:  0.4106, score:  0.8389
train_loss:  0.4067, valid_loss:  0.4136, score:  0.8359
train_loss:  0.3472, valid_loss:  0.7031, score:  0.7386
train_loss:  0.4112, valid_loss:  0.4649, score:  0.8298
train_loss:  0.3349, valid_loss

In [24]:
np.mean(score_list)

0.8532578060237634

# 5. 테스트 데이터 예측

In [25]:
test_dataset = FoodsDataset(test_file)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [26]:
pred_list = []
for i in range(n_splits):
    model = Net().to(device)
    model_params = torch.load(f"../output/model_{i}.pt", weights_only=True)
    model.load_state_dict(model_params)
    _, pred = test_loop(test_dataloader, model, loss_function, device)
    pred_list.append(pred)

pred = np.mean(pred_list, axis=0)
pred = np.argmax(pred, axis=1)
pred

array([1, 1, 1, ..., 1, 2, 1], dtype=int64)

# 6. 칸스 사이트의 컴피티션 페이지에 제출하여 점수 확인해보세요.

In [27]:
pd.DataFrame(pred, columns=["target"]).to_csv("../output/foods_classification_CNN.csv", index=False)